## Simple baseline with Landsat Cubes — ResNet6 + Binary Cross Entropy

To demonstrate the potential of a single-modality baseline with Landsat cubes, we provide a straightforward baseline based on a custom ResNet6-like architecture and Binary Cross-Entropy. The model itself should learn the relationship between the location's [R, G, B, NIR, SWIR1, and SWIR2] values and its species composition.

Considering the significant extent of enhancing the performance of this baseline, we encourage you to experiment with various techniques, architectures, losses, etc.

#### **Have Fun!**

In [1]:
%cd ..

/Users/23048869/Desktop/untitled folder/MMM_project


In [2]:
import os
import torch
import tqdm
import numpy as np
import pandas as pd
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
from sklearn.metrics import precision_recall_fscore_support
from torch.utils.data import Dataset

## Data description

Satellite time series data includes over 20 years of Landsat satellite imagery extracted from [Ecodatacube](https://stac.ecodatacube.eu/).
The data was acquired through the Landsat satellite program and pre-processed by Ecodatacube to produce raster files scaled to the entire European continent and projected into a unique CRS.

Since the original rasters require a high amount of disk space, we extracted the data points from each spectral band corresponding to all PA locations (i.e., GPS coordinates) and aggregated them in data cubes as tensor objects. Each data point corresponds to the mean value of Landsat's observations at the given location for three months before the given time. The cubes are structured as follows.
**Shape**: `(n_bands, n_quarters, n_years)` where:
- `n_bands` = 6 comprising [`red`, `green`, `blue`, `nir`, `swir1`, `swir2`]
- `n_quarters` = 4 
    - *Quarter 1*: December 2 of previous year until March 20 of current year (winter season proxy),
    - *Quarter 2*: March 21 until June 24 of current year (spring season proxy),
    - *Quarter 3*: June 25 until September 12 of current year (summer season proxy),
    - *Quarter 4*: September 13 until December 1 of current year (fall season proxy).
- `n_years` = 21 (ranging from 2000 to 2020)

The datacubes can simply be loaded as tensors using PyTorch with the following command :

```python
import torch
torch.load('path_to_file.pt')
```

**References:**
- *Traceability (lineage): This dataset is a seasonally aggregated and gapfilled version of the Landsat GLAD analysis-ready data product presented by Potapov et al., 2020 ( https://doi.org/10.3390/rs12030426 ).*
- *Scientific methodology: The Landsat GLAD ARD dataset was aggregated and harmonized using the eumap python package (available at https://eumap.readthedocs.io/en/latest/ ). The full process of gapfilling and harmonization is described in detail in Witjes et al., 2022 (in review, preprint available at https://doi.org/10.21203/rs.3.rs-561383/v3 ).*
- *Ecodatacube.eu: Analysis-ready open environmental data cube for Europe (https://doi.org/10.21203/rs.3.rs-2277090/v3).*

## Prepare custom dataset loader

We have to sloightly update the Dataset to provide the relevant data in the appropriate format.

In [3]:
class TrainDataset(Dataset):
    def __init__(self, data_dir, metadata, subset, transform=None):
        self.subset = subset
        self.transform = transform
        self.data_dir = data_dir
        self.metadata = metadata
        self.metadata = self.metadata.dropna(subset="speciesId").reset_index(drop=True)
        self.metadata['speciesId'] = self.metadata['speciesId'].astype(int)
        self.label_dict = self.metadata.groupby('surveyId')['speciesId'].apply(list).to_dict()
        
        self.metadata = self.metadata.drop_duplicates(subset="surveyId").reset_index(drop=True)


    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        
        survey_id = self.metadata.surveyId[idx]
        sample = torch.nan_to_num(torch.load(os.path.join(self.data_dir, f"GLC25-PA-{self.subset}-landsat-time-series_{survey_id}_cube.pt"), weights_only=True))

        species_ids = self.label_dict.get(survey_id, [])  # Get list of species IDs for the survey ID
        label = torch.zeros(11255)  # Initialize label tensor
        for species_id in species_ids:
            label_id = species_id
            label[label_id] = 1  # Set the corresponding class index to 1 for each species

        # Ensure the sample is in the correct format for the transform
        if isinstance(sample, torch.Tensor):
            sample = sample.permute(1, 2, 0)  # Change tensor shape from (C, H, W) to (H, W, C)
            sample = sample.numpy()  # Convert tensor to numpy array

        if self.transform:
            sample = self.transform(sample)

        return sample, label, survey_id
    
class TestDataset(TrainDataset):
    def __init__(self, data_dir, metadata, subset, transform=None):
        self.subset = subset
        self.transform = transform
        self.data_dir = data_dir
        self.metadata = metadata
        
    def __getitem__(self, idx):
        
        survey_id = self.metadata.surveyId[idx]
        sample = torch.nan_to_num(torch.load(os.path.join(self.data_dir, f"GLC25-PA-{self.subset}-landsat_time_series_{survey_id}_cube.pt"), weights_only=True))

        if isinstance(sample, torch.Tensor):
            sample = sample.permute(1, 2, 0)  # Change tensor shape from (C, H, W) to (H, W, C)
            sample = sample.numpy()

        if self.transform:
            sample = self.transform(sample)

        return sample, survey_id

### Load metadata and prepare data loaders

In [4]:
# Dataset and DataLoader
batch_size = 64
transform = transforms.Compose([
    transforms.ToTensor()
])

# Load Training metadata


train_data_path = "data/SateliteTimeSeries-Landsat/cubes/PA-train/"
train_metadata_path = "data/GLC25_PA_metadata_train.csv"
train_metadata = pd.read_csv(train_metadata_path)
train_dataset = TrainDataset(train_data_path, train_metadata, subset="train", transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)

# Load Test metadata
test_data_path = "data/SateliteTimeSeries-Landsat/cubes/PA-test/"
test_metadata_path = "data/GLC25_PA_metadata_test.csv"
test_metadata = pd.read_csv(test_metadata_path)
test_dataset = TestDataset(test_data_path, test_metadata, subset="test", transform=transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

## Define and initialize the ModifiedResNet18 model

To utilize the landsat cubes, which have a shape of [6,4,21] (BANDs, QUARTERs, and YEARs), some minor adjustments must be made to the vanilla ResNet-18. It's important to note that this is just one method for ensuring compatibility with the unusual tensor shape, and experimentation is encouraged.

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BasicBlock(nn.Module):
    """2×(Conv3×3 + BN + ReLU) with identity skip; stride=1 (no downsample)."""
    def __init__(self, c: int):
        super().__init__()
        self.conv1 = nn.Conv2d(c, c, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1   = nn.BatchNorm2d(c)
        self.conv2 = nn.Conv2d(c, c, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2   = nn.BatchNorm2d(c)

    def forward(self, x):
        identity = x
        out = self.conv1(x); out = self.bn1(out); out = F.relu(out, inplace=True)
        out = self.conv2(out); out = self.bn2(out)
        out = out + identity
        out = F.relu(out, inplace=True)
        return out

class ResNet6(nn.Module):
    """
    ResNet-6 for Landsat cubes:
      - Input: [B, 6, 4, 21]  (bands, quarters, years)
      - Stem: 1×1 conv (channel mixing) + 3×3 conv (spatial-temporal)
      - 3 residual blocks (6 conv layers total)
      - GAP + MLP head -> logits for multi-label BCEWithLogitsLoss
    """
    def __init__(self, num_classes: int, stem_channels: int = 64, mlp_hidden: int = 512, p_drop: float = 0.1):
        super().__init__()
        # Per-sample normalization over [C,H,W]
        self.norm_input = nn.LayerNorm([6, 4, 21])

        # Two-step stem to (i) mix spectral bands, then (ii) capture local 2D structure
        self.stem1 = nn.Conv2d(6, stem_channels, kernel_size=1, stride=1, padding=0, bias=False)  # channel mixing
        self.stem2 = nn.Conv2d(stem_channels, stem_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.stem_bn = nn.BatchNorm2d(stem_channels)

        # 3 residual blocks (no downsampling; preserve 4×21)
        self.block1 = BasicBlock(stem_channels)
        self.block2 = BasicBlock(stem_channels)
        self.block3 = BasicBlock(stem_channels)

        # Global average pooling and head
        self.gap = nn.AdaptiveAvgPool2d(1)
        self.head = nn.Sequential(
            nn.Flatten(),                       # [B, C, 1, 1] -> [B, C]
            nn.LayerNorm(stem_channels),
            nn.Linear(stem_channels, mlp_hidden),
            nn.ReLU(inplace=True),
            nn.Dropout(p_drop),
            nn.Linear(mlp_hidden, num_classes)  # logits
        )

        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.ones_(m.weight); nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                nn.init.trunc_normal_(m.weight, std=0.02); nn.init.zeros_(m.bias)

    def forward(self, x):
        # x: [B, 6, 4, 21]
        x = self.norm_input(x)
        x = self.stem1(x)                  # [B, C=stem_channels, 4, 21]
        x = self.stem2(x)
        x = self.stem_bn(x)
        x = F.relu(x, inplace=True)

        x = self.block1(x)                 # 3 blocks × 2 conv = 6 conv layers
        x = self.block2(x)
        x = self.block3(x)

        x = self.gap(x)                    # [B, C, 1, 1]
        x = self.head(x)                   # [B, num_classes] (logits)
        return x

In [6]:
def set_seed(seed):
    # Set seed for Python's built-in random number generator
    torch.manual_seed(seed)
    # Set seed for numpy
    np.random.seed(seed)
    # Set seed for CUDA if available
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        # Set cuDNN's random number generator seed for deterministic behavior
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(69)

In [7]:
# Check if cuda is available
device = torch.device("cpu")

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("DEVICE = CUDA")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print("DEVICE = MPS")


num_classes = 11255 # Number of all unique classes within the PO and PA data.
model = ResNet6(num_classes).to(device)

DEVICE = MPS


## Training Loop

Nothing special, just a standard Pytorch training loop.

In [8]:
# Hyperparameters
learning_rate = 0.001
num_epochs = 5

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
scheduler = CosineAnnealingLR(optimizer, T_max=num_epochs)

In [9]:
class AsymmetricLoss(nn.Module):
    """Asymmetric Loss for multi-label classification"""
    def __init__(self, gamma_neg=4, gamma_pos=1, clip=0.05, eps=1e-8):
        super().__init__()
        self.gamma_neg = gamma_neg
        self.gamma_pos = gamma_pos
        self.clip = clip
        self.eps = eps

    def forward(self, x, y):
        x_sigmoid = torch.sigmoid(x)
        xs_pos = x_sigmoid
        xs_neg = 1 - x_sigmoid

        if self.clip is not None and self.clip > 0:
            xs_neg = (xs_neg + self.clip).clamp(max=1)

        los_pos = y * torch.log(xs_pos.clamp(min=self.eps))
        los_neg = (1 - y) * torch.log(xs_neg.clamp(min=self.eps))
        loss = los_pos + los_neg

        pt0 = xs_pos * y
        pt1 = xs_neg * (1 - y)
        pt = pt0 + pt1
        one_sided_gamma = self.gamma_pos * y + self.gamma_neg * (1 - y)
        one_sided_w = torch.pow(1 - pt, one_sided_gamma)
        loss *= one_sided_w

        return -loss.sum(dim=1).mean()

In [ ]:
import time
from tqdm import tqdm
import torch

print(f"Training for {num_epochs} epochs started.")
start_time = time.time()

for epoch in range(num_epochs):
    epoch_start = time.time()
    model.train()

    running_loss = 0.0
    pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch+1}/{num_epochs}")

    for batch_idx, (data, targets, _) in pbar:
        data = data.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()
        outputs = model(data)

        # criterion = torch.nn.BCEWithLogitsLoss()
        criterion = AsymmetricLoss()
        loss = criterion(outputs, targets)

        loss.backward()
        optimizer.step()

        # Update running loss
        running_loss += loss.item()
        avg_loss = running_loss / (batch_idx + 1)

        # Show loss in the tqdm bar
        pbar.set_postfix({
            "batch_loss": f"{loss.item():.4f}",
            "avg_loss": f"{avg_loss:.4f}"
        })

    scheduler.step()
    epoch_time = time.time() - epoch_start
    print(f"\nEpoch {epoch+1} finished in {epoch_time:.2f} seconds")
    print("Scheduler:", scheduler.state_dict())

# Save the trained model
model.eval()
torch.save(model.state_dict(), "resnet6-with-landsat-cubes.pth")

total_time = time.time() - start_time
print(f"Training completed in {total_time/60:.2f} minutes")

Training for 5 epochs started.


Epoch 1/5:   1%|          | 11/1391 [00:00<00:52, 26.42it/s, batch_loss=8.3186, avg_loss=8.2007]

## Test Loop

Again, nothing special, just a standard inference.

In [11]:
with torch.no_grad():
    all_predictions = []
    surveys = []
    top_k_indices = None
    for data, surveyID in tqdm(test_loader, total=len(test_loader)):

        data = data.to(device)
        
        outputs = model(data)
        predictions = torch.sigmoid(outputs).cpu().numpy()

        # Sellect top-25 values as predictions
        top_25 = np.argsort(-predictions, axis=1)[:, :25] 
        if top_k_indices is None:
            top_k_indices = top_25
        else:
            top_k_indices = np.concatenate((top_k_indices, top_25), axis=0)

        surveys.extend(surveyID.cpu().numpy())

# with torch.no_grad():
#     all_predictions = []
#     surveys = []
#     top_k_indices = []
#     for data, surveyID in tqdm(test_loader, total=len(test_loader)):

#         data = data.to(device)
        
#         outputs = model(data)
#         predictions = torch.sigmoid(outputs).cpu().numpy()

#         # Sellect top-25 values as predictions
#         top_25 = np.argsort(-predictions, axis=1)[:, :25] 
#         if top_k_indices is None:
#             top_k_indices = top_25
#         else:
#             top_k_indices = np.concatenate((top_k_indices, top_25), axis=0)

#         # indices_above_0_5_per_sample = [np.where(sample > 0.1)[0] for sample in predictions]
#         # top_k_indices.extend(indices_above_0_5_per_sample)

#         surveys.extend(surveyID.cpu().numpy())

100%|██████████| 231/231 [00:13<00:00, 16.99it/s]


## Save prediction file! 🎉🥳🙌🤗

In [12]:
data_concatenated = [' '.join(map(str, row)) for row in top_k_indices]

pd.DataFrame(
    {'surveyId': surveys,
     'predictions': data_concatenated,
    }).to_csv("submission.csv", index = False)

In [13]:
lengths = [
    len(x.split()) for x in data_concatenated
]

print("Median", np.median(lengths))
print("Mean", np.mean(lengths))
print("Max", np.max(lengths))
print("Min", np.min(lengths))


Median 25.0
Mean 25.0
Max 25
Min 25
